# Purpose
This notebook is written as a requirement for the lab [AI Language Technology](https://www.researchgate.net/project/AI-Language-Technology) (WiSe 2018) offered by the Fraunhofer IAIS. It explains our implementation of a unified model for word sense disambiguation published in the following paper - 

Chen, X., Liu, Z., & Sun, M. (2014). A unified model for word sense representation and disambiguation. In Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP) (pp. 1025-1035). Accessed at https://www.aclweb.org/anthology/D14-1110.

(**Note:** For the sake of explanation, in some sections, texts are copied from the above publication. If a text was fully copied, it is shown in a quote.)

# Scope
The paper describes the method of disambiguating word sense as a **3-stage** process,
1. Initializing word vectors and sense vectors
2. Performing word sense disambiguation
3. Learning sense vectors from relevant occurrences

Our scope of this lab is to only perform the disambiguation of word sense. Hence, we do not go into the 3rd stage.

# Implementation
(Full project with python code and results is [here](https://github.com/iftekhar-ahmed/wsd-nlp).)

We use 2 python packages - `nltk` & `numpy`. From `nltk` we use the wordnet, punkt & averaged_perceptron_tagger libraries. The last one is the parts of speech (pos) tagger. 

```python
import nltk
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn
import numpy as np
from numpy import dot
from numpy import average
from numpy.linalg import norm
```
## Stage 1: Initializing word vectors and sense vectors
There are 2 parts to this stage.
### Initializing word vectors
The paper uses [Skipgram](https://becominghuman.ai/how-does-word2vecs-skip-gram-work-f92e0525def4) to train the word vectors from large amounts of text data. Since we use [glove](https://nlp.stanford.edu/projects/glove/) and [dependency-based](https://levyomer.wordpress.com/2014/04/25/dependency-based-word-embeddings/) word embeddings for word vector representation, we skip the training. Below is a method to load the word vectors.
```python
def load_glove_vectors(glove_file):
    f = open(glove_file, 'r')
    vectors = {}
    for line in f:
        split_line = line.split()
        word = split_line[0]
        embedding = np.array([float(val) for val in split_line[1:]])
        vectors[word] = embedding
    f.close()
    return vectors
```
This returns a dictionary of words and their corresponding vector embedding.

(Our glove data can be found [here](https://nlp.stanford.edu/projects/glove/))
### Initializing sense vectors
As outlined in Initializing sense vectors in Section 2.2 of the paper, we only consider words that are either *noun*, *verb*, *adjective* or *adverb* as candidates for disambiguation. We filter out the correct parts of speech with the following method,

```python
def get_valid_pos_tag(tag):
    if tag.startswith('J') or tag.startswith('V') or tag.startswith('N') or tag.startswith('R'):
        return True
    return False
```

We could not compile an exhaustive list of tags for the 4 parts of speech (pos) we are interested in. There are many different forms of tags for verb, noun, adjective and adverb as we've explored in the pos-tagger library. For this reason, we went on to choose the pos tags that start with one of the 4 letters - J, V, N and R.

Each candidate word has one or more *sense* and each sense has a *definition* and one or more *examples* in wordnet. For each sense, the task is to find its cosine similarities to each of the words in its gloss; then only take those words that clear a similarity threshold.
```python
cosine_sim_threshold = 0.05 # can be experimented with values ranging between 0-1
```
We find the normalized dot product between a sense vector and a word vector in its gloss to find their cosine similarity.
```python
cos_sim = dot(gloss_word_vec, candidate_vec) / (norm(gloss_word_vec) * norm(candidate_vec))
if cos_sim > cosine_sim_threshold:
    word_vectors.append(gloss_word_vec)
```
Finally, the average of the vectors for these words is used as the initialization value of the sense vector.
```python
sense_vector = average(word_vectors, 0) # Here, 0 denotes the Axis along which to average the vector arrays
The method below returns all sense vectors for a candidate word -
```python
def get_word_sense_vectors(candidate):
    vectors = {}
    try:
        candidate_vec = glove[candidate]
    except Exception:
        return None
    for sense in wn.lemmas(candidate):
        gloss = [sense.synset().definition()]
        gloss.extend(sense.synset().examples())
        word_vectors = []
        for sentence in gloss:
            tokens = nltk.word_tokenize(sentence)
            pos_tags = nltk.pos_tag(tokens)
            for gloss_pos, tag in pos_tags:
                if get_valid_pos_tag(tag):
                    try:
                        gloss_word_vec = glove[gloss_pos]
                    except Exception:
                        continue
                    cos_sim = dot(gloss_word_vec, candidate_vec) / (norm(gloss_word_vec) * norm(candidate_vec))
                    if cos_sim > cosine_sim_threshold:
                        word_vectors.append(gloss_word_vec)
        if len(word_vectors) == 0:
            continue
        sense_vector = average(word_vectors, 0)
        vectors[sense] = sense_vector
    return vectors
```
## Stage 2: Performing Word Sense Disambiguation
In order to disambiguate all of the content words in a sentence, the authors of the paper designed 2 algorithms namely - L2R (left to right) & S2C (simple to complex). They describe their algorithms as followed -
> The main difference between L2R and S2C is the order of words when performing word sense disambiguation. When given a sentence, the L2R algorithm disambiguates the words from left to right (the natural order of a sentence), whereas the S2C algorithm disambiguates the words with fewer senses first. The main idea of S2C algorithm is that the words with fewer senses are easier to disambiguate, and the disambiguation result can be helpful to sambiguate the words with more senses.

For our implementation we chose **S2C**. There are **3 steps** to this.
### Context vector initialization
Similar to the initialization of sense vectors, the average of all of the content words’ vectors in a sentence as the initialization vector of context.
```python
context_vec = average(list(pos_vectors.values()), 0) # 0 denotes the axis of the vector arrays
```
Here, `pos_vectors` contains the content word vectors from input sentence.

We also define a threshold value to determine if a disambiguated sense vector can replace word vector in the given sentence so that the context vector is recalculated.
```python
score_margin_threshold = 0.1 # this is the value that worked best for the experiments run in the main research
```
### Ranking words
According to the algorithm, we rank the content words based on the ascending order of their number of senses.
```python
sorted_sense_vectors_collection = sorted(sorted_sense_vectors_collection.items(), key=lambda x: x[1])
```
Here, `sorted_sense_vectors_collection` contains a dictionary of content words and a list containing their corresponding sense vectors - before sorting. Then we pass it to the `sorted()` numpy method.
### Word sense disambiguation
For each content word in the `sorted_sense_vectors_collection`, we compute the cosine similarities between its sense vectors and the context vector. We choose the sense that yields the maximum cosine similarity as its disambiguation result. We also find the cosine similarity score margin between the nearest sense (disambiguated sense) and second nearest sense. The method looks like this,
```python
def disambiguate_word_sense(word, context_vector):
    vectors = sense_vectors_collection[word]
    if len(vectors) == 0:
        return [None, 0.0]
    cos_sims = {}
    for sense, sense_vector in vectors.items():
        cos_sim = dot(context_vector, sense_vector) / (norm(context_vector) * norm(sense_vector))
        cos_sims[sense] = cos_sim
    sorted_list = sorted(cos_sims.items(), key=lambda x: x[1])
    if len(sorted_list) == 0:
        return [None, 0.0]
    most_similar_pair = sorted_list.pop()
    disambiguated_sense = most_similar_pair[0]
    cos_sim_second_most_similar_sense = 0
    if len(sorted_list) > 0:
        cos_sim_second_most_similar_sense = sorted_list.pop()[1]
    score_margin = most_similar_pair[1] - cos_sim_second_most_similar_sense
    # we return the disambiguated sense AND the cosine score margin between the two most similar senses.
    return [disambiguated_sense, score_margin]
```
From the above method we get the disambiguated sense and score margin. We look at the score margin and if it is greater than the threshold we defined above, we use the disambiguated sense vector to replace the word vector in `pos_vectors`. Then we repeat **Context vector initialization** to update the context vector.
```python
if score_margin > score_margin_threshold:
    pos_vectors[w] = sense_vectors_collection[content_word][disambiguated_sense]
    context_vec = average(list(pos_vectors.values()), 0)
```
From here, we feed the updated context vector along with the next content word to the method. This updated context vector helps better disambiguate the next content word. We repeat this entire step until all content words are disambiguated.

# Experiment
For detailed information on how each step in the algorithm works when we run the experiment, click [here](https://nbviewer.jupyter.org/github/logicalforhad/word-sense-disambiguation/blob/master/word_sense.ipynb).

## Dataset
Our dataset is taken from the following paper -

Koeling, Rob, Diana McCarthy, and John Carroll. "Domain-specific sense distributions and predominant sense acquisition." Proceedings of the conference on Human Language Technology and Empirical Methods in Natural Language Processing. Association for Computational Linguistics, 2005. Accessed at https://www.researchgate.net/publication/220816751_Domain-Specific_Sense_Distributions_and_Predominant_Sense_Acquisition.

Dataset can be accessed [here](http://www.dianamccarthy.co.uk/downloads/hlt2005releasev2.tgz).

Our dataset contains 41 different content words in 3 different categories - banking & commerce, sports and finance. For each of the 41 words our dataset contains around 100 examples under each category. That sums to ~300 sentences for each content word to run experiment on.

## Experiment Design

### Loading annotations
We start by loading the annotated WordNet sense keys from the `gold_standard_clean.txt` file in our dataset. For each sentence in the dataset, there is a linkup key provided to link its disambiguation result to the result file. For each linkup key, the dataset contains 4 sense keys annotated by 4 distinct authors. We store the sense keys against their linkup key in a dictionary.
```python
def load_annotations():
    path = "/home/iftekhar/Desktop/gold/gold_standard_clean.txt"
    with open(path, 'r', encoding='ISO-8859-1') as f:
        for lines in f:
            line = lines.split('|')
            if len(line) < 4:
                continue
            linkup_key = line[1].strip()
            wn_key = line[2].strip()
            wn_sense_keylist = list()
            if linkup_key in annotation_results:
                wn_sense_keylist = annotation_results[linkup_key]
            else:
                annotation_results[linkup_key] = wn_sense_keylist
            if wn_key == "unclear":
                continue
            wn_sense_keylist.append(wn_key)
```
The results are stored in the `annotation_results` dictionary.

### Running the experiment
We define another method takes a sentence and a lookup word as parameters and returns the key in WordNet for the disambiguated sense (if found) for the word.
```python
def find_wn_sense_key(sentence, lookup_word):
```
The next steps are to look for content words in the input sentence, find sense vectors for each content word, initializing the context vector and ranking the content words - as we've described in the **Implementation** section. After the disambiguation method returns for each content word we do the following,

If the content word is the lookup word, we have found the disambiguated sense. We simply return the sense key as our disambiguation result.
```python
if content_word == lookup_word:
    wn_sense_key = disambiguated_sense.key()
    break
return wn_sense_key
```
Otherwise we update the context vector based on the score margin and repeat the disambiguation process for the next content word.

We match the disambiguated sense key against the most frequent sense key in `annotation_results` dictionary - linking the sentence file and dictionary through the linkup key we mentioned. If the two keys match, we consider our dismambiguation result correct.
```python
if most_frequent_annotated_wn_sense_key == wn_sense_key:
    output_file.write("correct wsd for " + linkup_key + " " + wn_sense_key + "\n")
    correct_count += 1
    output_file.write("correct " + str(correct_count) + " | total " + str(total_sentence_count) + "\n")
else:
    output_file.write("incorrect wsd for " + linkup_key + " | found " + wn_sense_key + ", correct is " + most_frequent_annotated_wn_sense_key + "\n")
```

For the given dataset, we run the experiment for 4 different word embeddings.

## Results
A summary of our results is given below:

* total __11427__ valid examples were executed for word sense disambiguation
* total __683__ invalid examples identified in gold standard clean dataset
* overall __12110__ examples were executed and dumped to output file

For each of the 4 word embeddings we used, the number of correctly disambiguated senses and the recall value (calculated by dividing the number of correctly disambiguated word senses by the total number of valid examples in gold standard dataset) are listed below.

| Name                                                 | Correct Disambiguation | Recall    |
|:-----------------------------------------------------|:-----------------------|:----------|
| [Glove](https://tinyurl.com/y4dtgcg7)                | 2760                   | **24.15** |
| [Dependency-Based](https://tinyurl.com/y4czh4j7)     | 2790                   | **24.41** |
| [Bag of Words (k = 2)](https://tinyurl.com/yyv5vkfa) | 2774                   | **24.27** |
| [Bag of Words (k = 5)](https://tinyurl.com/yxw28j4g) | 2813                   | **24.61** |

For each word embedding, the corresponding output dump and disambiguation results are listed below.

| Name                 | Output                                 | Results                                 |
|:---------------------|:---------------------------------------|:----------------------------------------|
| Glove                | [Output](https://tinyurl.com/yyl5nxup) | [Results](https://tinyurl.com/y2onoys3) |
| Dependency-Based     | [Output](https://tinyurl.com/y5rjdfga) | [Results](https://tinyurl.com/y39wpuv7) |
| Bag of Words (k = 2) | [Output](https://tinyurl.com/y4slfr6c) | [Results](https://tinyurl.com/yy67bmld) |
| Bag of Words (k = 5) | [Output](https://tinyurl.com/y4nrcua7) | [Results](https://tinyurl.com/yyvrhc9g) |
